# 2.Marketing_Case-ELVT-Build

In [1]:
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sns
import numpy as np
from collections import Counter
import pickle

import pprint
import operator

import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

from tabulate import tabulate

In [3]:
FILE = 'marketing_training_cleaned.csv'
TEST_DF_FILE = './marketing_test.pkl'
MODEL_FILE = 'marketing_model'
data = pd.read_csv(FILE)

In [4]:
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.101-b13, mixed mode, sharing)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\gsaez\AppData\Local\Temp\tmplgnckx79
  JVM stdout: C:\Users\gsaez\AppData\Local\Temp\tmplgnckx79\h2o_gsaez_started_from_python.out
  JVM stderr: C:\Users\gsaez\AppData\Local\Temp\tmplgnckx79\h2o_gsaez_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.4
H2O cluster version age:,3 months and 29 days !!!
H2O cluster name:,H2O_from_python_gsaez_8zjyfj
H2O cluster total nodes:,1
H2O cluster free memory:,247.5 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [5]:
mrk = h2o.H2OFrame(data)
mrk.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:7380
Cols:21




,cons.conf.idx,cons.price.idx,contact,day_of_week,default,emp.var.rate,euribor3m,housing,loan,marital,month,nr.employed,poutcome,profession,responded,schooling,campaign1,custAge1,pastEmail1,pcontacted_pdays,pContacted_previous
type,real,real,enum,enum,enum,real,real,enum,enum,enum,enum,real,enum,enum,enum,enum,int,int,enum,enum,enum
mins,-50.8,92.201,,,,-3.4,0.634,,,,,4963.6,,,,,1.0,18.0,,,
mean,-40.56775067750686,93.57061829268248,,,,0.048265582655826506,3.5789429539295465,,,,,5165.0473712737175,,,,,2.0264026402640316,39.80514905149058,,,
maxs,-26.9,94.767,,,,1.4,5.045,,,,,5228.1,,,,,6.0,64.0,,,
sigma,4.653732975650036,0.5791390659374276,,,,1.5697319890162231,1.7462251198691165,,,,,73.14368800240456,,,,,1.2618634459509646,8.93268712954785,,,
zeros,0,0,,,,0,0,,,,,0,,,,,0,0,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,411,0,0,0,0
0,-42.0,93.2,cellular,mon,unknown,-0.1,4.191,no,no,single,nov,5195.8,nonexistent,admin.,no,university.degree,1.0,55.0,no,True,no
1,-42.7,93.918,cellular,mon,no,1.4,4.96,no,no,married,jul,5228.1,nonexistent,blue-collar,no,other,1.0,42.0,no,True,no
2,-36.4,93.994,telephone,mon,no,1.1,4.857,no,no,married,may,5191.0,nonexistent,technician,no,high.school,1.0,42.0,no,True,no


In [6]:
categorical_variables = [] 

for variable in mrk.types.items():
    if variable[1] == 'enum':
        categorical_variables.append(variable[0])

for variable in categorical_variables:
    mrk[variable] = mrk[variable].asfactor() 
    
predictors = mrk.columns
predictors.remove('responded')
response = 'responded'


train, valid, oot = mrk.split_frame(ratios=[.8,.1], seed=25)

#save TEST df for future testing
oot[1,:].as_data_frame().to_pickle(TEST_DF_FILE)

In [7]:
def check_model(model, show_all = False):
    if show_all:
        display(model)
        
    print ("full model training auc:", model.auc())
    print ("full model cv auc:", model.auc(xval=True))
    for model_ in model.get_xval_models():
        print (model_.model_id, " training auc:",model_.auc(), " validation auc: ", model_.auc(valid=True))

## Binary model (GLM)

In [8]:
MRK_binomial = H2OGeneralizedLinearEstimator(family = "binomial", nfolds = 10)
MRK_binomial.train(y = response, x = predictors, training_frame = train)

check_model(MRK_binomial, True)

glm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1543594182307_1


ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.07729023145762855
RMSE: 0.2780112074316943
LogLoss: 0.2722304619003809
Null degrees of freedom: 5921
Residual degrees of freedom: 5870
Null deviance: 4164.729370979201
Residual deviance: 3224.297590748111
AIC: 3328.297590748111
AUC: 0.8077650995744603
Gini: 0.6155301991489206
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.23821849421784894: 


,no,yes,Error,Rate
no,4809.0,447.0,0.085,(447.0/5256.0)
yes,314.0,352.0,0.4715,(314.0/666.0)
Total,5123.0,799.0,0.1285,(761.0/5922.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2382185,0.4805461,189.0
max f2,0.0995309,0.5542618,271.0
max f0point5,0.4386026,0.5307263,103.0
max accuracy,0.4482634,0.9044242,100.0
max precision,0.9229888,1.0,0.0
max recall,0.0122205,1.0,397.0
max specificity,0.9229888,1.0,0.0
max absolute_mcc,0.2646409,0.4118028,175.0
max min_per_class_accuracy,0.0810474,0.7378234,287.0
max mean_per_class_accuracy,0.0995309,0.7475781,271.0


Gains/Lift Table: Avg response rate: 11.25 %, avg score: 11.25 %



...


Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
accuracy,0.875325,0.0165855,0.8913044,0.8571429,0.8768972,0.9026403,0.9018612,0.8409476,0.8667883,0.9093904,0.8475121,0.8587663
auc,0.7941319,0.0234411,0.7523693,0.8023974,0.7666466,0.8413824,0.7940683,0.7666925,0.8537562,0.7644705,0.7781132,0.8214228
err,0.1246750,0.0165855,0.1086957,0.1428571,0.1231029,0.0973597,0.0981387,0.1590525,0.1332117,0.0906096,0.1524880,0.1412338
err_count,73.9,10.418493,60.0,85.0,73.0,59.0,58.0,94.0,73.0,55.0,95.0,87.0
f0point5,0.4754098,0.0367892,0.53125,0.4684096,0.4571429,0.5191257,0.5805243,0.4246284,0.486618,0.4444444,0.3984064,0.4435484
---,---,---,---,---,---,---,---,---,---,---,---,---
r2,0.2030915,0.0317450,0.2174239,0.1856742,0.1719722,0.2744812,0.2362755,0.1842527,0.2699509,0.1255563,0.1625553,0.2027727
recall,0.5350406,0.0747387,0.53125,0.5733333,0.4848485,0.6551724,0.4366197,0.5333334,0.5970149,0.2857143,0.6060606,0.6470588
residual_deviance,331.3939,19.88174,322.70935,363.72293,342.61395,277.1499,339.1357,373.9014,294.09732,318.56177,348.39923,333.64743
rmse,0.2816223,0.0094924,0.2832201,0.2995121,0.2861838,0.2505855,0.2841265,0.3006393,0.2799016,0.270612,0.2816369,0.2798048



See the whole table with table.as_data_frame()
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2018-11-30 10:09:53,0.000 sec,0,2082.3646855,0.3516320
,2018-11-30 10:09:53,0.015 sec,1,1780.3305221,0.3031733
,2018-11-30 10:09:53,0.015 sec,2,1649.4113581,0.2804791
,2018-11-30 10:09:53,0.034 sec,3,1613.6208563,0.2741455
,2018-11-30 10:09:53,0.034 sec,4,1612.1487954,0.2740209


full model training auc: 0.8077650995744603
full model cv auc: 0.7922075900101013
GLM_model_python_1543594182307_1_cv_1  training auc: 0.8129372310419407  validation auc:  0.7523693647540983
GLM_model_python_1543594182307_1_cv_2  training auc: 0.8072441492888828  validation auc:  0.802397435897436
GLM_model_python_1543594182307_1_cv_3  training auc: 0.8094630647776133  validation auc:  0.766646541314473
GLM_model_python_1543594182307_1_cv_4  training auc: 0.8037330775164334  validation auc:  0.841382456581928
GLM_model_python_1543594182307_1_cv_5  training auc: 0.806847249034749  validation auc:  0.794068255687974
GLM_model_python_1543594182307_1_cv_6  training auc: 0.8082101779862494  validation auc:  0.7666925064599484
GLM_model_python_1543594182307_1_cv_7  training auc: 0.8016975937207738  validation auc:  0.853756167189003
GLM_model_python_1543594182307_1_cv_8  training auc: 0.8086711032908834  validation auc:  0.7644704433497537
GLM_model_python_1543594182307_1_cv_9  training auc:

In [9]:
MRK_binomial_pred = MRK_binomial.predict(oot)

MRK_binomial_pred.cbind(oot['responded'])

glm prediction progress: |████████████████████████████████████████████████| 100%


predict,no,yes,responded
no,0.971928,0.028072,no
no,0.848528,0.151472,no
no,0.967886,0.0321143,no
no,0.955439,0.0445614,no
no,0.872358,0.127642,no
no,0.946033,0.0539668,no
no,0.9702,0.0297996,no
no,0.958961,0.0410386,no
no,0.924152,0.0758479,no
no,0.975278,0.0247217,no


## Random Forest (RF)

In [10]:
MRK_RF = H2ORandomForestEstimator(nfolds = 10)
MRK_RF.train(y = response, x = predictors, training_frame = train, validation_frame = valid)

check_model(MRK_RF, True)

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1543594182307_35


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.08595006826310624
RMSE: 0.29317242070683636
LogLoss: 0.4205646876145264
Mean Per-Class Error: 0.2877226541610103
AUC: 0.7697150632367528
Gini: 0.5394301264735055
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3040482502751306: 


,no,yes,Error,Rate
no,4833.0,423.0,0.0805,(423.0/5256.0)
yes,362.0,304.0,0.5435,(362.0/666.0)
Total,5195.0,727.0,0.1326,(785.0/5922.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3040483,0.4364681,172.0
max f2,0.1014787,0.5055698,289.0
max f0point5,0.4381496,0.4694049,121.0
max accuracy,0.6775794,0.8946302,55.0
max precision,0.8947368,0.8695652,9.0
max recall,0.0000015,1.0,399.0
max specificity,1.0,0.9996195,0.0
max absolute_mcc,0.3420388,0.3663427,157.0
max min_per_class_accuracy,0.1014787,0.7085236,289.0
max mean_per_class_accuracy,0.1594344,0.7122773,247.0


Gains/Lift Table: Avg response rate: 11.25 %, avg score: 12.26 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-11-30 10:10:12,16.720 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-11-30 10:10:12,16.770 sec,1.0,0.3923793,4.7958531,0.6146979,2.7934705,0.1659732,0.3802179,4.5441553,0.6327664,3.1571429,0.1652661
,2018-11-30 10:10:12,16.796 sec,2.0,0.3708937,3.9378682,0.6381372,3.3531714,0.1618364,0.3323616,2.1922566,0.7040627,5.1,0.1666667
,2018-11-30 10:10:12,16.812 sec,3.0,0.3609259,3.5526463,0.6453998,3.3746337,0.1629328,0.3142846,1.5474346,0.7274943,5.6666667,0.1274510
,2018-11-30 10:10:12,16.856 sec,4.0,0.3601626,3.3302326,0.6488862,3.3311807,0.1729414,0.3032044,1.2193395,0.7576625,5.3125,0.1414566
---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2018-11-30 10:10:14,19.106 sec,46.0,0.2932743,0.4477716,0.7677736,5.7797297,0.1339075,0.2830093,0.3074988,0.7784014,5.3125,0.0924370
,2018-11-30 10:10:14,19.175 sec,47.0,0.2931391,0.4424253,0.7683193,5.8801221,0.1413374,0.2837160,0.3082149,0.7771070,5.3125,0.0952381
,2018-11-30 10:10:14,19.253 sec,48.0,0.2931870,0.4371939,0.7685091,5.9279279,0.1445458,0.2834327,0.3086292,0.7749811,6.375,0.0938375
,2018-11-30 10:10:14,19.327 sec,49.0,0.2930671,0.4308617,0.7703264,6.2243243,0.1354272,0.2835557,0.3071361,0.7754157,6.375,0.0938375



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
custAge1,1886.4661865,1.0,0.1100216
profession,1751.2375488,0.9283164,0.1021348
day_of_week,1547.3743896,0.8202503,0.0902452
euribor3m,1497.1154785,0.7936084,0.0873141
schooling,1395.2908936,0.7396321,0.0813755
nr.employed,1205.3436279,0.6389426,0.0702975
campaign1,1012.2072754,0.5365626,0.0590335
poutcome,923.8111572,0.4897046,0.0538781
month,884.4044800,0.4688154,0.0515798
pcontacted_pdays,817.5466919,0.4333747,0.0476806


full model training auc: 0.7697150632367528
full model cv auc: 0.7665042325430453
DRF_model_python_1543594182307_35_cv_1  training auc: 0.7648803494532763  validation auc:  0.7655242417005015
DRF_model_python_1543594182307_35_cv_2  training auc: 0.7462026119873996  validation auc:  0.7723656968508227
DRF_model_python_1543594182307_35_cv_3  training auc: 0.7639745529855934  validation auc:  0.7472260748959778
DRF_model_python_1543594182307_35_cv_4  training auc: 0.7604581623781056  validation auc:  0.7501756756756757
DRF_model_python_1543594182307_35_cv_5  training auc: 0.757200872735996  validation auc:  0.7760999793431109
DRF_model_python_1543594182307_35_cv_6  training auc: 0.7644266867256759  validation auc:  0.7270114942528736
DRF_model_python_1543594182307_35_cv_7  training auc: 0.7531003812122733  validation auc:  0.7695402298850574
DRF_model_python_1543594182307_35_cv_8  training auc: 0.752938511507811  validation auc:  0.7839023057046314
DRF_model_python_1543594182307_35_cv_9  

In [11]:
MRK_RF = MRK_RF.predict(oot)
MRK_RF.cbind(oot['responded'])

drf prediction progress: |████████████████████████████████████████████████| 100%


predict,no,yes,responded
no,0.953772,0.0462276,no
yes,0.551032,0.448968,no
no,0.932387,0.0676126,no
no,0.966839,0.0331613,no
no,0.66893,0.33107,no
no,0.934565,0.0654349,no
no,0.964009,0.0359913,no
no,0.979187,0.0208132,no
no,0.898801,0.101199,no
no,0.966429,0.0335714,no


## Gradient boosting machine model (GBM)

In [12]:
mrk_gbm = H2OGradientBoostingEstimator(nbins_cats = 10, seed = 25)
mrk_gbm.train(x = predictors, y= response, training_frame = train, validation_frame = valid)

print ("full model training auc:", mrk_gbm.auc())
print ("full model cv auc:", mrk_gbm.auc(xval=True))

mrk_gbm

gbm Model Build progress: |███████████████████████████████████████████████| 100%
full model training auc: 0.8735059260173414
full model cv auc: None
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1543594182307_439


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.059138631115738034
RMSE: 0.24318435623151838
LogLoss: 0.22092790865445622
Mean Per-Class Error: 0.20859815294746797
AUC: 0.8735059260173414
Gini: 0.7470118520346829
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3709257430983883: 


,no,yes,Error,Rate
no,5172.0,84.0,0.016,(84.0/5256.0)
yes,315.0,351.0,0.473,(315.0/666.0)
Total,5487.0,435.0,0.0674,(399.0/5922.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3709257,0.6376022,153.0
max f2,0.1735784,0.6258850,236.0
max f0point5,0.4525717,0.7587065,127.0
max accuracy,0.4454011,0.9341439,129.0
max precision,0.9613578,1.0,0.0
max recall,0.0263788,1.0,390.0
max specificity,0.9613578,1.0,0.0
max absolute_mcc,0.3942940,0.6232927,145.0
max min_per_class_accuracy,0.0807657,0.7751142,306.0
max mean_per_class_accuracy,0.1735784,0.7914018,236.0


Gains/Lift Table: Avg response rate: 11.25 %, avg score: 11.26 %



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0112045,0.7837158,6.375,6.375,0.75,0.8621315,0.75,0.8621315,0.0714286,0.0714286,537.5,537.5
,2,0.0210084,0.7153593,7.2857143,6.8000000,0.8571429,0.7318412,0.8,0.8013293,0.0714286,0.1428571,628.5714286,580.0000000
,3,0.0308123,0.6628411,4.8571429,6.1818182,0.5714286,0.6904931,0.7272727,0.7660633,0.0476190,0.1904762,385.7142857,518.1818182
,4,0.0406162,0.5768848,7.2857143,6.4482759,0.8571429,0.6095449,0.7586207,0.7282830,0.0714286,0.2619048,628.5714286,544.8275862
,5,0.0504202,0.4841981,4.8571429,6.1388889,0.5714286,0.5235029,0.7222222,0.6884646,0.0476190,0.3095238,385.7142857,513.8888889
,6,0.1008403,0.2308080,3.5416667,4.8402778,0.4166667,0.3522154,0.5694444,0.5203400,0.1785714,0.4880952,254.1666667,384.0277778
,7,0.1498599,0.1542216,1.4571429,3.7336449,0.1714286,0.1927428,0.4392523,0.4131820,0.0714286,0.5595238,45.7142857,273.3644860
,8,0.2002801,0.1180378,0.9444444,3.0314685,0.1111111,0.1374603,0.3566434,0.3437696,0.0476190,0.6071429,-5.5555556,203.1468531
,9,0.2997199,0.0709518,1.3169014,2.4626168,0.1549296,0.0907670,0.2897196,0.2598295,0.1309524,0.7380952,31.6901408,146.2616822
,10,0.4005602,0.0563943,0.1180556,1.8723776,0.0138889,0.0623303,0.2202797,0.2101094,0.0119048,0.75,-88.1944444,87.2377622



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-11-30 10:10:16,0.022 sec,0.0,0.3159340,0.3516320,0.5,1.0,0.8875380,0.3222315,0.3623435,0.5,1.0,0.8823529
,2018-11-30 10:10:17,0.113 sec,1.0,0.3067249,0.3277873,0.8059795,8.0729018,0.1247889,0.3145895,0.3414754,0.7684429,5.6666667,0.1008403
,2018-11-30 10:10:17,0.144 sec,2.0,0.2999140,0.3135652,0.8095324,8.1234568,0.1165147,0.3091884,0.3290025,0.7965325,5.3125,0.1274510
,2018-11-30 10:10:17,0.166 sec,3.0,0.2940544,0.3025882,0.8118365,8.1234568,0.1014860,0.3038098,0.3178115,0.8000189,5.3125,0.0966387
,2018-11-30 10:10:17,0.182 sec,4.0,0.2890906,0.2938570,0.8197231,8.3144963,0.1001351,0.2995698,0.3096862,0.7988757,5.3125,0.0924370
---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2018-11-30 10:10:18,1.494 sec,46.0,0.2449626,0.2235936,0.8706719,8.8918919,0.0680513,0.2778973,0.2759777,0.7929138,7.4375,0.1176471
,2018-11-30 10:10:18,1.535 sec,47.0,0.2444988,0.2229804,0.8710017,8.8918919,0.0687268,0.2782948,0.2766610,0.7921391,7.4375,0.1036415
,2018-11-30 10:10:18,1.569 sec,48.0,0.2443182,0.2227339,0.8712860,8.8918919,0.0683891,0.2784638,0.2768931,0.7918084,6.375,0.1008403
,2018-11-30 10:10:18,1.603 sec,49.0,0.2435339,0.2214310,0.8729581,8.8918919,0.0673759,0.2782424,0.2766586,0.7923658,6.375,0.1050420



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
nr.employed,370.6296997,1.0,0.3256646
month,132.7046661,0.3580519,0.1166048
day_of_week,114.7292709,0.3095523,0.1008102
euribor3m,97.3329163,0.2626150,0.0855244
schooling,72.7561493,0.1963042,0.0639293
poutcome,71.6351471,0.1932796,0.0629443
custAge1,68.1525955,0.1838833,0.0598843
pcontacted_pdays,64.7792130,0.1747815,0.0569202
profession,29.1151371,0.0785559,0.0255829
marital,20.2725792,0.0546977,0.0178131


In [13]:
mrk_gbm_pred = mrk_gbm.predict(oot)
mrk_gbm_pred.cbind(oot['responded'])

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,no,yes,responded
no,0.974409,0.0255911,no
yes,0.663645,0.336355,no
no,0.965264,0.0347365,no
no,0.945047,0.0549526,no
no,0.827919,0.172081,no
no,0.936581,0.0634186,no
no,0.966882,0.0331177,no
no,0.931096,0.0689035,no
no,0.967808,0.032192,no
no,0.964552,0.0354482,no


## Save Model

In [14]:
# save the model
model_path = h2o.save_model(model = mrk_gbm, path = MODEL_FILE, force = True)

# Shutdown h2o

In [15]:
h2o.cluster().shutdown()

H2O session _sid_886c closed.
